In [2]:
import pandas as pd
from elasticsearch import Elasticsearch
from datetime import datetime

In [3]:
from config import ELASTIC_PASSWORD,CERT_PATH

client = Elasticsearch(
    "https://localhost:9200",
    ca_certs=CERT_PATH,
    basic_auth=("elastic",ELASTIC_PASSWORD)
)

client.info()

ObjectApiResponse({'name': 'eneko-VirtualBox', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'WqGfVbtESlmYTx_o7tckkg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
#Variables seleccionadas
VARIABLES_SELECCIONADAS = [
    #Fechas
    'booked_at', 'checkin_date',
    # Meteorologia
    'lead_time', 'lenght_of_stay', 'estacion_estancia',
    #Cliente
    'adult_count', 'child_count', 'origin', 
    #Negocio/Ingresos
    'rate', 'total_adr',
    #Estado reserva
    'status', 'ha_cancelado', 'cancellation_reason', 
    'zona_roja', 'momento_checkin',
    #Geografía
    'asset', 'city', 'provincia',
    #Meteorología 
    'tmed', 'prec', 'tmin', 'tmax', 
    'sol', 'hrMedia', 'velmedia'
]

In [5]:
#ÍNDICE FINAL
INDEX_NAME = "hotel_analytics_final"

#MAPPING EXCLUSIVO
mapping_final = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            # FECHAS
            "booked_at": {"type": "date", "format": "strict_date_optional_time"},
            "checkin_date": {"type": "date", "format": "yyyy-MM-dd"},
            # ENTEROS
            "lead_time": {"type": "integer"},
            "lenght_of_stay": {"type": "integer"},
            "adult_count": {"type": "integer"},
            "child_count": {"type": "integer"},
            # KEYWORDS
            "estacion_estancia": {"type": "keyword"},
            "origin": {"type": "keyword"},
            "status": {"type": "keyword"},
            "cancellation_reason": {"type": "keyword"},
            "momento_checkin": {"type": "keyword"},
            "asset": {"type": "keyword"},
            "city": {"type": "keyword"},
            "provincia": {"type": "keyword"},  
            "province_code": {"type": "keyword"},
            # FLOATS
            "rate": {"type": "float"},
            "total_adr": {"type": "float"},
            "tmed": {"type": "float"},
            "prec": {"type": "float"},
            "tmin": {"type": "float"},
            "tmax": {"type": "float"},
            "sol": {"type": "float"},
            "hrMedia": {"type": "float"},
            "velmedia": {"type": "float"},
            # BOOLEANS
            "ha_cancelado": {"type": "boolean"},
            "zona_roja": {"type": "boolean"},
            "coordinates": {"type": "geo_point"}
        }
    }
}

#Eliminar el índice si ya existe
if client.indices.exists(index=INDEX_NAME):
    client.indices.delete(index=INDEX_NAME)
    print(" Índice anterior eliminado")

#Crear el índice con el mapping definido
client.indices.create(
    index=INDEX_NAME,
    body=mapping_final
)

print(f" Índice '{INDEX_NAME}' creado con {len(mapping_final['mappings']['properties'])} campos")



 Índice anterior eliminado


/tmp/ipykernel_7186/3065307768.py:54: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.indices.create(


 Índice 'hotel_analytics_final' creado con 27 campos


In [7]:
df_es = pd.read_csv(
    "/home/eneko/Documentos/RETO_06/datos/reservas_final.csv",
    encoding="latin-1"
)
df_es


,booked_at,checkin_time,checkout_time,lead_time,lenght_of_stay,checkin_month,checkin_day,adult_count,child_count,origin,...,tmax,dir,velmedia,racha,sol,presMax,presMin,hrMedia,hrMax,hrMin
0,2022-11-26,2023-01-01,2023-01-02,36,1,January,Sunday,1,0,channel_manager,...,24.6,24.000000,4.700000,18.9,3.150000,1029.2,1013.4,53.500000,92.0,19.0
1,2022-12-21,2023-01-01,2023-01-10,11,9,January,Sunday,1,0,channel_manager,...,24.6,18.500000,3.720000,18.9,3.790000,1034.6,1005.4,70.700000,97.0,19.0
2,2022-09-21,2023-01-01,2023-01-07,102,6,January,Sunday,2,4,channel_manager,...,24.6,17.428571,3.528571,18.9,4.357143,1034.6,1012.0,66.571429,96.0,19.0
3,2022-09-24,2023-01-01,2023-01-02,99,1,January,Sunday,2,2,channel_manager,...,24.6,24.000000,4.700000,18.9,3.150000,1029.2,1013.4,53.500000,92.0,19.0
4,2022-10-18,2023-01-01,2023-01-02,75,1,January,Sunday,4,0,channel_manager,...,24.6,24.000000,4.700000,18.9,3.150000,1029.2,1013.4,53.500000,92.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52094,2023-10-10,2023-12-31,2024-01-01,82,1,December,Sunday,4,0,channel_manager,...,12.2,26.500000,1.550000,8.3,4.250000,967.0,959.9,76.000000,96.0,51.0
52095,2023-10-13,2023-12-31,2024-01-01,79,1,December,Sunday,4,0,telephone,...,12.2,26.500000,1.550000,8.3,4.250000,967.0,959.9,76.000000,96.0,51.0
52096,2023-10-17,2023-12-31,2024-01-01,75,1,December,Sunday,5,0,direct_channel,...,12.2,26.500000,1.550000,8.3,4.250000,967.0,959.9,76.000000,96.0,51.0
52097,2023-10-17,2023-12-31,2024-01-01,75,1,December,Sunday,4,0,direct_channel,...,12.2,26.500000,1.550000,8.3,4.250000,967.0,959.9,76.000000,96.0,51.0


In [12]:
#Asegurarse de que fechas son datetime
df_es["booked_at"] = pd.to_datetime(df_es["booked_at"], errors="coerce")
df_es["checkin_time"] = pd.to_datetime(df_es["checkin_time"], errors="coerce") 

#Añadir columna provincia
map_ciudad_provincia = {
    "Donostia": "Gipuzkoa",
    "Vitoria": "Álava",
    "Bilbao": "Bizkaia",
    "Valencia": "Valencia",
    "Madrid": "Madrid",
    "Málaga": "Málaga",
    "Granada": "Granada",
    "Córdoba": "Córdoba",
    "Pamplona": "Navarra"
}
df_es["provincia"] = df_es["city"].map(map_ciudad_provincia)

#Codigos ISO
PROVINCE_ISO_MAP = {
    "Bizkaia": "ES-BI",
    "Madrid": "ES-M",
    "Granada": "ES-GR",
    "Valencia": "ES-V",
    "Gipuzkoa": "ES-SS",
    "Álava": "ES-VI",
    "Navarra": "ES-NA",
    "Málaga": "ES-MA",
    "Córdoba": "ES-CO"
}

#Funciones
def safe_float(x):
    try:
        if pd.isna(x):
            return None
        return float(x)
    except (ValueError, TypeError):
        return None

def safe_bool(x):
    if pd.isnull(x):
        return None
    if isinstance(x, bool):
        return x
    if str(x).lower() in ["true", "1"]:
        return True
    return False

#Indexar
for i, data in df_es.iterrows():
    doc = dict()
    
    #Fechas ISO
    doc["booked_at"] = data["booked_at"].isoformat() if pd.notnull(data["booked_at"]) else None
    doc["checkin_time"] = data["checkin_time"].strftime("%Y-%m-%d") if pd.notnull(data["checkin_time"]) else None
    
    #Enteros
    doc["lead_time"] = int(data["lead_time"]) if pd.notnull(data["lead_time"]) else None
    doc["lenght_of_stay"] = int(data["lenght_of_stay"]) if pd.notnull(data["lenght_of_stay"]) else None
    doc["adult_count"] = int(data["adult_count"]) if pd.notnull(data["adult_count"]) else None
    doc["child_count"] = int(data["child_count"]) if pd.notnull(data["child_count"]) else None
    
    #Keywords
    for col in ["estacion_estancia","origin","status","cancellation_reason",
                "momento_checkin","asset","city","provincia"]:
        doc[col] = str(data[col]) if pd.notnull(data[col]) else None

    #Codigo ISO nuevo
    doc["province_code"] = PROVINCE_ISO_MAP.get(data["provincia"], None)

    #Floats
    for col in ["rate","total_adr","tmed","prec","tmin","tmax","sol","hrMedia","velmedia"]:
        doc[col] = safe_float(data[col])
    
    #Booleanos
    doc["ha_cancelado"] = safe_bool(data["ha_cancelado"])
    doc["zona_roja"] = safe_bool(data["zona_roja"])
    
    #Indexar
    try:
        client.index(index=INDEX_NAME, id=i+1, document=doc)
    except Exception as e:
        print(f" Error indexando documento {i+1}: {e}")
        print(f"Documento problema: {doc}")
        break

print(f" {len(df_es)} documentos indexados en '{INDEX_NAME}'")


KeyboardInterrupt: 

In [15]:
#Verificar que los documentos existen
# Contar documentos en el índice
client.count(index=INDEX_NAME)


ObjectApiResponse({'count': 52099, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [16]:
# Obtener el primer documento
res = client.get(index=INDEX_NAME, id=1)
print(res["_source"])


{'booked_at': '2022-11-26T00:00:00', 'checkin_date': '2023-01-01', 'lead_time': 36, 'lenght_of_stay': 1, 'adult_count': 1, 'child_count': 0, 'estacion_estancia': 'invierno', 'origin': 'channel_manager', 'status': 'checked_out', 'cancellation_reason': 'not cancelled', 'momento_checkin': 'antes', 'asset': 'Koisi Hostel', 'city': 'Donostia', 'provincia': 'Gipuzkoa', 'province_code': 'ES-SS', 'rate': None, 'total_adr': 15.92, 'tmed': 16.75, 'prec': 0.95, 'tmin': 10.3, 'tmax': 24.6, 'sol': 3.15, 'hrMedia': 53.5, 'velmedia': 4.7, 'ha_cancelado': False, 'zona_roja': True}
